In [1]:
# import libraries.
import torch
import numpy as np
import datetime
# tf.enable_eager_execution()
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data.sampler import Sampler
from torch.autograd import Variable
from torch.optim.lr_scheduler import MultiplicativeLR, StepLR, LambdaLR
import h5py
import glob
import matplotlib.pyplot as plt
import time
import os
import torch.optim as optim
from natsort import natsorted
print('imported')
# #######################



imported


# Caution! change foldername before execution

In [2]:
# Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
# Data_path = 'D:/hale'
# Data_path = '/content/drive/My Drive/thesis/datasets'
Data_path = os.getcwd()[0:-5]
ckpt_folder = '1'

checkpoint_path = os.path.normpath(os.path.join(Data_path,'results','checkpoints',ckpt_folder))
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
files = glob.glob(os.path.normpath(os.path.join(checkpoint_path,'*')))
for f in files:
    os.remove(f)
    
loss_path = os.path.normpath(os.path.join(Data_path,'results','losses',ckpt_folder))
if not os.path.exists(loss_path):
    os.makedirs(loss_path)
files = glob.glob(os.path.normpath(os.path.join(loss_path,'*')))
for f in files:
    os.remove(f)
    
time_path = os.path.normpath(os.path.join(Data_path,'results','times',ckpt_folder))
if not os.path.exists(time_path):
    os.makedirs(time_path)
files = glob.glob(os.path.normpath(os.path.join(time_path,'*')))
for f in files:
    os.remove(f)

    
timestep = 16
batch_size = 32
n_features = 257
epochs_num =100
lr = 0.0001

In [3]:
class Dataset_loader():

    def __init__(self, path, batch_size, timestep, X_, Y_):
        self.write_path = path
        self.batch_size = batch_size
        self.len_data = 0
        self.X_ = X_
        self.Y_ = Y_
        self.timestep = timestep
        with h5py.File(os.path.normpath(os.path.join(self.write_path))+'.hdf5', "r") as f:
            obj = f['X']
            self.len_data = obj.shape[0]
            self.n_features = obj.shape[1]
        self.index_arr = np.arange(self.len_data)
        self.index_arr = self.index_arr[0:(len(self.index_arr)//self.timestep)*self.timestep]
        self.index_arr = self.index_arr.reshape((len(self.index_arr)//self.timestep,self.timestep))
        np.random.shuffle(self.index_arr)

    def check_if_left(self):
        if len(self.index_arr)>=self.batch_size:
            return True
        return False

    def get_batch(self):
        self.X_out=self.X_[self.index_arr[0:self.batch_size].reshape(1,self.batch_size*self.timestep)]
        self.Y_out=self.Y_[self.index_arr[0:self.batch_size].reshape(1,self.batch_size*self.timestep)]
        self.index_arr = self.index_arr[self.batch_size:]
        return self.X_out.reshape(self.batch_size,self.timestep,self.n_features),self.Y_out.reshape(self.batch_size,self.timestep,self.n_features)

In [4]:
path = os.path.normpath(os.path.join(Data_path,'h5_files_e','30hdata_2','features','data_30h_loge'))
path_val = os.path.normpath(os.path.join(Data_path,'h5_files_e','devdata_3','features','devdata_30h_loge'))
files = 'data_30h_loge'
files_val = 'devdata_30h_loge'
with h5py.File(path+'.hdf5', "r") as f:
    X = f['X'][:]
with h5py.File(path+'.hdf5', "r") as f:
    Y = f['Y'][:]
with h5py.File(path_val+'.hdf5', "r") as f:
    X_val = f['X'][:]
with h5py.File(path_val+'.hdf5', "r") as f:
    Y_val = f['Y'][:]

In [5]:
load_batch = Dataset_loader( path, batch_size, timestep, X, Y)
load_batch.check_if_left()

True

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(input_size=n_features,hidden_size=257, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(514, 257)
    def forward(self, x):
        x, _  = self.lstm(x)
        x = self.fc(x)
        return x
    
net = Net()
net.cuda()
net.train()

Net(
  (lstm): LSTM(257, 257, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=514, out_features=257, bias=True)
)

In [15]:
def custom_loss(y_true, y_pred, epoch, w1, w2, al1, al2, fbank):
        batch_size = y_true.shape[0]
        n_features = y_true.shape[-1]
        true = torch.sqrt(torch.exp(y_true))
        pred = torch.sqrt(torch.exp(y_pred))
        fbank= fbank.reshape((1,fbank.shape[0],fbank.shape[1]))
        fbank = np.repeat(self.fbank, self.batch_size, axis=0)
        filtered_true = torch.einsum('mnp,mqp->mnq', true, torch.tensor(self.fbank,device=torch.device('cuda:0')))
        filtered_pred = torch.einsum('mnp,mqp->mnq', self.pred, torch.tensor(self.fbank,device=torch.device('cuda:0')))
#         print(torch.mean(torch.pow(self.filtered_true-self.filtered_pred,2)))
#         print(torch.mean(torch.pow(self.y_true-self.y_pred,2)))
        l1 = torch.mean(torch.pow(self.filtered_true-self.filtered_pred,2))*w1
        l2 = torch.mean(torch.pow(self.y_true-self.y_pred,2))*w2
        l = (torch.mean(torch.pow(self.filtered_true-self.filtered_pred,2))*w1)+(w2*torch.mean(torch.pow(self.y_true-self.y_pred,2)))
        return l, l1, l2

optimizer = optim.Adam(net.parameters(), lr=lr)

In [16]:
lambda1 = lambda epoch: 0.9**(epoch-10) if epoch>10 else 1
scheduler = LambdaLR(optimizer, lr_lambda=lambda1)

In [17]:

train_loss = np.zeros((epochs_num,1))
val_loss = np.zeros((epochs_num,1))+np.inf
start = time.time()
l_1 = []
l_2 = []
for epoch in range(epochs_num): 
    running_loss = 0.0
    loss_tot = 0.0
    v_loss_tot = 0.0
    i=0
    load_batch = Dataset_loader(path, batch_size, timestep, X, Y)
    load_batch_val = Dataset_loader(path_val, batch_size, timestep, X_val, Y_val)
    net.train()
    while load_batch.check_if_left():
        
        inputs, labels = load_batch.get_batch()
        
#         print(stop-start)
        inputs=Variable(torch.tensor(inputs).float())
        labels=Variable(torch.tensor(labels).float())
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        optimizer.zero_grad()
        outputs = net(inputs)
        criterion = custom_losses(outputs, labels, epoch)
        loss = criterion()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        loss_tot += loss.item()
        if (i+1) % 1000 == 0:   
            print('[epoch %d, minibatch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0
        i = i+1
    train_loss[epoch] = loss_tot/i
    print('epoch %d, train loss: %.3f' %(epoch+1, loss_tot/i))
    net.eval()
    i=0
    while load_batch_val.check_if_left():
        inputs_val, labels_val = load_batch_val.get_batch()
        inputs_val=Variable(torch.tensor(inputs_val).float())
        labels_val=Variable(torch.tensor(labels_val).float())
        inputs_val = inputs_val.cuda()
        labels_val = labels_val.cuda()
        outputs_val = net(inputs_val)
        criterion = custom_losses(outputs_val, labels_val, epoch)
        v_loss = criterion()
        v_loss_tot += v_loss.item()
        i=i+1
    print(optimizer.param_groups[0]['lr'])
#     scheduler.step()
    if (v_loss_tot/i)<np.min(val_loss):
        torch.save(net.state_dict(), os.path.normpath(os.path.join(checkpoint_path,'ckpt_'+str(epoch+1)+'.pth')))
    val_loss[epoch] = v_loss_tot/i
    print('************ epoch %d, val loss: %.3f ************' %(epoch+1, v_loss_tot/i))
print('Finished Training')
stop = time.time()
with open((os.path.normpath(os.path.join(time_path,str(ckpt_folder)+'.txt'))), 'w') as f:
  f.write('%d' % int(stop-start))
np.savetxt(os.path.normpath(os.path.join(loss_path,'train_loss.txt')),train_loss)
np.savetxt(os.path.normpath(os.path.join(loss_path,'val_loss.txt')),val_loss)
model_path = os.path.normpath(os.path.join(Data_path,'results','models',str(ckpt_folder)+'.pth'))
torch.save(net.state_dict(), model_path)
print('Finished saving')

[epoch 1, minibatch  1000] loss: 52.441
[epoch 1, minibatch  2000] loss: 24.978
[epoch 1, minibatch  3000] loss: 21.178
[epoch 1, minibatch  4000] loss: 18.993
[epoch 1, minibatch  5000] loss: 17.390
[epoch 1, minibatch  6000] loss: 16.084
[epoch 1, minibatch  7000] loss: 15.057
[epoch 1, minibatch  8000] loss: 14.292
[epoch 1, minibatch  9000] loss: 13.613
[epoch 1, minibatch 10000] loss: 13.230
[epoch 1, minibatch 11000] loss: 12.737
[epoch 1, minibatch 12000] loss: 12.523
[epoch 1, minibatch 13000] loss: 12.244
[epoch 1, minibatch 14000] loss: 11.839
epoch 1, train loss: 17.908
0.0001
************ epoch 1, val loss: 19.962 ************
[epoch 2, minibatch  1000] loss: 11.215
[epoch 2, minibatch  2000] loss: 11.109
[epoch 2, minibatch  3000] loss: 10.794
[epoch 2, minibatch  4000] loss: 10.628
[epoch 2, minibatch  5000] loss: 10.644
[epoch 2, minibatch  6000] loss: 10.451
[epoch 2, minibatch  7000] loss: 10.175
[epoch 2, minibatch  8000] loss: 10.055
[epoch 2, minibatch  9000] loss: 

epoch 13, train loss: 4.860
0.0001
************ epoch 13, val loss: 12.913 ************
[epoch 14, minibatch  1000] loss: 4.674
[epoch 14, minibatch  2000] loss: 4.741
[epoch 14, minibatch  3000] loss: 4.686
[epoch 14, minibatch  4000] loss: 4.688
[epoch 14, minibatch  5000] loss: 4.826
[epoch 14, minibatch  6000] loss: 4.690
[epoch 14, minibatch  7000] loss: 4.731
[epoch 14, minibatch  8000] loss: 4.730
[epoch 14, minibatch  9000] loss: 4.765
[epoch 14, minibatch 10000] loss: 4.686
[epoch 14, minibatch 11000] loss: 4.728
[epoch 14, minibatch 12000] loss: 4.752
[epoch 14, minibatch 13000] loss: 4.733
[epoch 14, minibatch 14000] loss: 4.705
epoch 14, train loss: 4.729
0.0001
************ epoch 14, val loss: 13.568 ************
[epoch 15, minibatch  1000] loss: 4.581
[epoch 15, minibatch  2000] loss: 4.565
[epoch 15, minibatch  3000] loss: 4.630
[epoch 15, minibatch  4000] loss: 4.585
[epoch 15, minibatch  5000] loss: 4.601
[epoch 15, minibatch  6000] loss: 4.566
[epoch 15, minibatch  70

[epoch 26, minibatch 10000] loss: 3.744
[epoch 26, minibatch 11000] loss: 3.794
[epoch 26, minibatch 12000] loss: 3.782
[epoch 26, minibatch 13000] loss: 3.744
[epoch 26, minibatch 14000] loss: 3.772
epoch 26, train loss: 3.753
0.0001
************ epoch 26, val loss: 12.399 ************
[epoch 27, minibatch  1000] loss: 3.640
[epoch 27, minibatch  2000] loss: 3.659
[epoch 27, minibatch  3000] loss: 3.675
[epoch 27, minibatch  4000] loss: 3.669
[epoch 27, minibatch  5000] loss: 3.684
[epoch 27, minibatch  6000] loss: 3.723
[epoch 27, minibatch  7000] loss: 3.701
[epoch 27, minibatch  8000] loss: 3.709
[epoch 27, minibatch  9000] loss: 3.764
[epoch 27, minibatch 10000] loss: 3.727
[epoch 27, minibatch 11000] loss: 3.749
[epoch 27, minibatch 12000] loss: 3.699
[epoch 27, minibatch 13000] loss: 3.699
[epoch 27, minibatch 14000] loss: 3.718
epoch 27, train loss: 3.703
0.0001
************ epoch 27, val loss: 12.430 ************
[epoch 28, minibatch  1000] loss: 3.607
[epoch 28, minibatch  20

[epoch 39, minibatch  5000] loss: 3.316
[epoch 39, minibatch  6000] loss: 3.278
[epoch 39, minibatch  7000] loss: 3.277
[epoch 39, minibatch  8000] loss: 3.298
[epoch 39, minibatch  9000] loss: 3.290
[epoch 39, minibatch 10000] loss: 3.280
[epoch 39, minibatch 11000] loss: 3.313
[epoch 39, minibatch 12000] loss: 3.338
[epoch 39, minibatch 13000] loss: 3.304
[epoch 39, minibatch 14000] loss: 3.301
epoch 39, train loss: 3.302
0.0001
************ epoch 39, val loss: 11.867 ************
[epoch 40, minibatch  1000] loss: 3.220
[epoch 40, minibatch  2000] loss: 3.242
[epoch 40, minibatch  3000] loss: 3.267
[epoch 40, minibatch  4000] loss: 3.241
[epoch 40, minibatch  5000] loss: 3.293
[epoch 40, minibatch  6000] loss: 3.249
[epoch 40, minibatch  7000] loss: 3.255
[epoch 40, minibatch  8000] loss: 3.293
[epoch 40, minibatch  9000] loss: 3.257
[epoch 40, minibatch 10000] loss: 3.280
[epoch 40, minibatch 11000] loss: 3.247
[epoch 40, minibatch 12000] loss: 3.294
[epoch 40, minibatch 13000] loss

0.0001
************ epoch 51, val loss: 11.714 ************
[epoch 52, minibatch  1000] loss: 3.004
[epoch 52, minibatch  2000] loss: 2.993
[epoch 52, minibatch  3000] loss: 3.052
[epoch 52, minibatch  4000] loss: 3.080
[epoch 52, minibatch  5000] loss: 3.025
[epoch 52, minibatch  6000] loss: 3.038
[epoch 52, minibatch  7000] loss: 3.003
[epoch 52, minibatch  8000] loss: 3.020
[epoch 52, minibatch  9000] loss: 3.025
[epoch 52, minibatch 10000] loss: 3.067
[epoch 52, minibatch 11000] loss: 3.060
[epoch 52, minibatch 12000] loss: 3.024
[epoch 52, minibatch 13000] loss: 3.113
[epoch 52, minibatch 14000] loss: 3.054
epoch 52, train loss: 3.042
0.0001
************ epoch 52, val loss: 11.602 ************
[epoch 53, minibatch  1000] loss: 3.007
[epoch 53, minibatch  2000] loss: 2.995
[epoch 53, minibatch  3000] loss: 2.980
[epoch 53, minibatch  4000] loss: 3.044
[epoch 53, minibatch  5000] loss: 3.014
[epoch 53, minibatch  6000] loss: 3.061
[epoch 53, minibatch  7000] loss: 3.036
[epoch 53, m

[epoch 64, minibatch 10000] loss: 2.906
[epoch 64, minibatch 11000] loss: 2.895
[epoch 64, minibatch 12000] loss: 2.896
[epoch 64, minibatch 13000] loss: 2.929
[epoch 64, minibatch 14000] loss: 2.921
epoch 64, train loss: 2.899
0.0001
************ epoch 64, val loss: 11.447 ************
[epoch 65, minibatch  1000] loss: 2.817
[epoch 65, minibatch  2000] loss: 2.864
[epoch 65, minibatch  3000] loss: 2.907
[epoch 65, minibatch  4000] loss: 2.856
[epoch 65, minibatch  5000] loss: 2.880
[epoch 65, minibatch  6000] loss: 2.854
[epoch 65, minibatch  7000] loss: 2.878
[epoch 65, minibatch  8000] loss: 2.886
[epoch 65, minibatch  9000] loss: 2.857
[epoch 65, minibatch 10000] loss: 2.905
[epoch 65, minibatch 11000] loss: 2.957
[epoch 65, minibatch 12000] loss: 2.960
[epoch 65, minibatch 13000] loss: 2.862
[epoch 65, minibatch 14000] loss: 2.879
epoch 65, train loss: 2.882
0.0001
************ epoch 65, val loss: 11.100 ************
[epoch 66, minibatch  1000] loss: 2.811
[epoch 66, minibatch  20

[epoch 77, minibatch  5000] loss: 2.747
[epoch 77, minibatch  6000] loss: 2.750
[epoch 77, minibatch  7000] loss: 2.764
[epoch 77, minibatch  8000] loss: 2.779
[epoch 77, minibatch  9000] loss: 2.814
[epoch 77, minibatch 10000] loss: 2.765
[epoch 77, minibatch 11000] loss: 2.781
[epoch 77, minibatch 12000] loss: 2.805
[epoch 77, minibatch 13000] loss: 2.799
[epoch 77, minibatch 14000] loss: 2.788
epoch 77, train loss: 2.776
0.0001
************ epoch 77, val loss: 11.838 ************
[epoch 78, minibatch  1000] loss: 2.761
[epoch 78, minibatch  2000] loss: 2.749
[epoch 78, minibatch  3000] loss: 2.724
[epoch 78, minibatch  4000] loss: 2.750
[epoch 78, minibatch  5000] loss: 2.751
[epoch 78, minibatch  6000] loss: 2.784
[epoch 78, minibatch  7000] loss: 2.737
[epoch 78, minibatch  8000] loss: 2.738
[epoch 78, minibatch  9000] loss: 2.770
[epoch 78, minibatch 10000] loss: 2.791
[epoch 78, minibatch 11000] loss: 2.756
[epoch 78, minibatch 12000] loss: 2.771
[epoch 78, minibatch 13000] loss

0.0001
************ epoch 89, val loss: 11.380 ************
[epoch 90, minibatch  1000] loss: 2.653
[epoch 90, minibatch  2000] loss: 2.645
[epoch 90, minibatch  3000] loss: 2.692
[epoch 90, minibatch  4000] loss: 2.698
[epoch 90, minibatch  5000] loss: 2.655
[epoch 90, minibatch  6000] loss: 2.691
[epoch 90, minibatch  7000] loss: 2.673
[epoch 90, minibatch  8000] loss: 2.706
[epoch 90, minibatch  9000] loss: 2.669
[epoch 90, minibatch 10000] loss: 2.803
[epoch 90, minibatch 11000] loss: 2.690
[epoch 90, minibatch 12000] loss: 2.688
[epoch 90, minibatch 13000] loss: 2.677
[epoch 90, minibatch 14000] loss: 2.782
epoch 90, train loss: 2.695
0.0001
************ epoch 90, val loss: 11.358 ************
[epoch 91, minibatch  1000] loss: 2.630
[epoch 91, minibatch  2000] loss: 2.665
[epoch 91, minibatch  3000] loss: 2.693
[epoch 91, minibatch  4000] loss: 2.660
[epoch 91, minibatch  5000] loss: 2.664
[epoch 91, minibatch  6000] loss: 2.710
[epoch 91, minibatch  7000] loss: 2.674
[epoch 91, m

In [ ]:
train_loss.data.cpu().numpy().reshape(1,1)[0]